In [10]:
import pandas as pd
import numpy as np
import seaborn as sns

In [11]:
data = pd.read_csv('train_PDjVQMB.csv')

In [12]:
data.head()

,ID,Age,Gender,Income,Balance,Vintage,Transaction_Status,Product_Holdings,Credit_Card,Credit_Category,Is_Churn
0,84e2fcc9,36,Female,5L - 10L,563266.44,4,0,1,0,Average,1
1,57fea15e,53,Female,Less than 5L,875572.11,2,1,1,1,Poor,0
2,8df34ef3,35,Female,More than 15L,701607.06,2,1,2,0,Poor,0
3,c5c0788b,43,Female,More than 15L,1393922.16,0,1,2,1,Poor,1
4,951d69c4,39,Female,More than 15L,893146.23,1,1,1,1,Good,1


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6650 entries, 0 to 6649
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  6650 non-null   object 
 1   Age                 6650 non-null   int64  
 2   Gender              6650 non-null   object 
 3   Income              6650 non-null   object 
 4   Balance             6650 non-null   float64
 5   Vintage             6650 non-null   int64  
 6   Transaction_Status  6650 non-null   int64  
 7   Product_Holdings    6650 non-null   object 
 8   Credit_Card         6650 non-null   int64  
 9   Credit_Category     6650 non-null   object 
 10  Is_Churn            6650 non-null   int64  
dtypes: float64(1), int64(5), object(5)
memory usage: 571.6+ KB


In [14]:
data.corr()

,Age,Balance,Vintage,Transaction_Status,Credit_Card,Is_Churn
Age,1.000000,0.002154,0.017353,0.032166,0.012089,0.204301
Balance,0.002154,1.000000,-0.019385,-0.005088,-0.025259,0.056380
Vintage,0.017353,-0.019385,1.000000,-0.027286,0.001856,0.020152
Transaction_Status,0.032166,-0.005088,-0.027286,1.000000,0.006526,-0.086917
Credit_Card,0.012089,-0.025259,0.001856,0.006526,1.000000,-0.000848
Is_Churn,0.204301,0.056380,0.020152,-0.086917,-0.000848,1.000000


In [15]:
import pandas_profiling
from pandas_profiling import ProfileReport
profile = ProfileReport(data, explorative=True, dark_mode=True)
profile.to_file('output.html')

In [17]:
sns.countplot(x="Vintage",hue="Is_Churn",data=data)

<AxesSubplot:xlabel='Vintage', ylabel='count'>

In [4]:
dropped_column=["Gender"]

In [5]:
data.drop(columns=dropped_column,inplace=True)

In [6]:
dataset=data

In [7]:
id_col=data.pop('ID')

In [8]:
data['Age_cat']=data.Age

In [9]:
data['bal_age']=(data.Age)/data.Balance

In [10]:
data.shape

(6650, 11)

In [11]:
categorical=list(set(['Gender','Income','Transaction_Status','Vintage','Age_cat','Product_Holdings','Credit_Card','Credit_Category'])-set(dropped_column))

In [12]:
target=['Is_Churn']

In [13]:
continuous=list(set(list(data.columns))-set(categorical)-set(target))

In [14]:
continuous

['Age', 'bal_age', 'Balance']

In [15]:
len(categorical)

7

## Dataset preprocessing

In [16]:
data=pd.get_dummies(dataset,columns=categorical)

In [17]:
data.shape

(6650, 76)

In [18]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler,QuantileTransformer,Normalizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,roc_auc_score
from sklearn.svm import LinearSVC,SVC
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [19]:
scale=QuantileTransformer(n_quantiles=975,output_distribution="uniform")#MinMaxScaler(feature_range=(0,1))

In [20]:
data.shape

(6650, 76)

In [21]:
for col in continuous:
    scale.fit(data[col].values.reshape(-1,1))
    data[col]=scale.transform(data[col].values.reshape(-1,1))

In [22]:
data.shape

(6650, 76)

In [23]:
Y=data.pop('Is_Churn')

In [24]:
X=data

In [25]:
Y.value_counts()

0    5113
1    1537
Name: Is_Churn, dtype: int64

### IMBlearn Undersampling 

In [26]:
cats=np.arange(3,len(X.columns))
cats

array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
       71, 72, 73, 74])

In [29]:
from imblearn.under_sampling import EditedNearestNeighbours,AllKNN,ClusterCentroids,NearMiss,CondensedNearestNeighbour,TomekLinks
from imblearn.over_sampling import SVMSMOTE,BorderlineSMOTE,ADASYN,SMOTE,KMeansSMOTE,SMOTENC
from imblearn.ensemble import BalancedBaggingClassifier

enn = EditedNearestNeighbours(n_neighbors=2,n_jobs=-1,kind_sel="all",sampling_strategy="majority")
X_resampled, y_resampled = enn.fit_resample(X,Y)


In [30]:
y_resampled.value_counts()

0    3175
1    1537
Name: Is_Churn, dtype: int64

In [32]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(y_resampled),y=y_resampled.values)

class_weights = dict(zip(np.unique(y_resampled),class_weights))
class_weights

{0: 0.7420472440944882, 1: 1.5328562134027326}

## Model fitting

In [33]:
from sklearn.linear_model import LogisticRegression,BayesianRidge,Lasso
from sklearn.model_selection import RandomizedSearchCV
from sklearn.naive_bayes import MultinomialNB,CategoricalNB,BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as lda

from sklearn.svm import SVC,NuSVC
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
from xgboost import XGBClassifier,XGBRFClassifier
from sklearn.linear_model import SGDClassifier,ElasticNetCV
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier,AdaBoostClassifier,StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.02, random_state=42)

In [249]:
DecisionTreeClassifier

sklearn.tree._classes.DecisionTreeClassifier

In [35]:
dtc=DecisionTreeClassifier(max_depth=1)
clf=AdaBoostClassifier(dtc,n_estimators=50,learning_rate=1)

In [36]:
clf.fit(X_train, y_train)
preds=clf.predict(X_test)

In [37]:
accuracy_score(preds,y_test)

0.7789473684210526

In [38]:
pd.DataFrame(classification_report(preds,y_test,output_dict=True)).T

,precision,recall,f1-score,support
0,0.953846,0.775000,0.855172,80.000000
1,0.400000,0.800000,0.533333,15.000000
accuracy,0.778947,0.778947,0.778947,0.778947
macro avg,0.676923,0.787500,0.694253,95.000000
weighted avg,0.866397,0.778947,0.804356,95.000000


In [39]:
y_test.value_counts()

0    65
1    30
Name: Is_Churn, dtype: int64

In [40]:
pd.value_counts(preds)

0    80
1    15
dtype: int64

In [41]:
clf.fit(X_resampled, y_resampled)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=1)

## Evaluate on standard test data

In [42]:
test = pd.read_csv('test_lTY72QC.csv')

In [43]:
categorical

['Income',
 'Vintage',
 'Credit_Card',
 'Transaction_Status',
 'Product_Holdings',
 'Credit_Category',
 'Age_cat']

In [45]:
data.Balance.describe()

count    6650.000000
mean        0.500000
std         0.288746
min         0.000000
25%         0.249763
50%         0.499998
75%         0.750113
max         1.000000
Name: Balance, dtype: float64

In [46]:
test_ID=test.pop("ID")

In [47]:
test['Age_cat']=test.Age
test['bal_age']=(test.Age)/test.Balance


In [48]:
test.drop(columns=dropped_column,inplace=True)

In [49]:
len(categorical)

7

In [50]:
test.shape

(2851, 10)

In [51]:
test=pd.get_dummies(test,columns=categorical)


In [52]:
test.shape

(2851, 75)

In [53]:
continuous

['Age', 'bal_age', 'Balance']

In [55]:
for col in continuous:
    scale.fit(test[col].values.reshape(-1,1))
    test[col]=scale.transform(test[col].values.reshape(-1,1))

In [56]:
test.shape

(2851, 75)

In [57]:
data.shape

(6650, 75)

In [58]:
predicted=clf.predict(test)


In [59]:
pd.value_counts(predicted)

0    2257
1     594
dtype: int64

In [281]:
solution=pd.DataFrame(test_ID)
# solution

In [282]:
target

['Is_Churn']

In [283]:
solution[target]=predicted

In [284]:
np.arange(1,len(solution))

array([   1,    2,    3, ..., 2848, 2849, 2850])

In [285]:
solution.set_index(np.arange(1,len(solution)+1),inplace=True)

In [286]:
# solution.Is_Churn=0

In [287]:
solution.to_csv("../../submission/AAAABB submission_41.csv",index=False)

In [288]:
solution.shape

(2851, 2)